In [1]:
# import os
# import json
import boto3
# import pandas as pd
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import train_test_split

import sagemaker
# from sagemaker.inputs import TrainingInput
# from sagemaker.xgboost.estimator import XGBoost
# from sagemaker.debugger import Rule, rule_configs
# from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

# from sagemaker import clarify
# from sagemaker.session import Session


In [2]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity()["Account"]

random_state = 42

Using AWS Region: us-east-1


In [3]:
%store -r
%store

Stored variables and their in-db values:
clarify_data_uri                    -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
data_prefix                         -> 'sagemaker-tutorial/data'
default_bucket                      -> 'sagemaker-us-east-1-367158743199'
feature_group_name                  -> 'FG-flow-sm-tutorial-31-16-16-17-9f41d66b'
hyperparameters                     -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
local_processed_data                -> '../sm-tutorial/02_build_train/processed_data'
model_data                          -> 's3://sagemaker-us-east-1-367158743199/tf2-resnet-
mp2_arn                             -> 'arn:aws:sagemaker:us-east-1:367158743199:model-pa
mpg_name                            -> 'sagemaker-tutorial'
prefix                              -> 'sagemaker-tutorial'
s3_raw_data                         -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
s3_train_data                       -> 's3://sagemaker-us-east-1-367158743199/sage

# Approve the Second model

In [4]:
second_model_package = sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]
model_package_update = {
    'ModelPackageArn': second_model_package['ModelPackageArn'],
    'ModelApprovalStatus': 'Approved'
}

update_response = sagemaker_boto_client.update_model_package(**model_package_update)

In [11]:
second_model_package

{'ModelPackageGroupName': 'sagemaker-tutorial',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:367158743199:model-package/sagemaker-tutorial/2',
 'ModelPackageDescription': 'XGBoost classifier with SMOTE',
 'CreationTime': datetime.datetime(2021, 4, 2, 2, 32, 30, 40000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'PendingManualApproval'}

# Create an endpoint

In [10]:
approved_model_name = 'xgboost-mnist'

endpoint_name = f'{approved_model_name}-endpoint'
endpoint_instance_count = 1
endpoint_instance_type = "ml.m4.xlarge"


primary_container = {'ModelPackageName': second_model_package['ModelPackageArn']}
endpoint_config_name=f'{approved_model_name}-endpoint-config'
existing_configs = len(sagemaker_boto_client.list_endpoint_configs(NameContains=endpoint_config_name, MaxResults = 30)['EndpointConfigs'])

if existing_configs == 0:
    create_ep_config_response = sagemaker_boto_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': endpoint_instance_type,
            'InitialVariantWeight': 1,
            'InitialInstanceCount': endpoint_instance_count,
            'ModelName': approved_model_name,
            'VariantName': 'AllTraffic'
        }]
    )
    %store endpoint_config_name
    % approved_model_name

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Could not find model "arn:aws:sagemaker:us-east-1:367158743199:model/xgboost-mnist".

In [ ]:
arn:aws:sagemaker:us-east-1:367158743199:model-package/sagemaker-tutorial/2